# Data Analysis - Cleaning Data
### Loans Originated in 2018
### Predict/Classifying what types of loans/borrowers will be 
    Comparing previous months behavior between current and delinquent borrowers
    - paid up to current [~1e6]
    - become delinquent [51,115]
    
    # # 
    - paid off in full (may not work) [with zero_balance_code "1": 220,906] 
    - become default 
    
    # #
    # Split Train/Test to CSV
    models (SVC, LogReg, RandomForest, # Boosting)
    RandomSearchCV
    
    Future work:
    Looking at previous 2-6 months windows

Create Path to Import *.py files (not needed in "index.ipynb")

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# sys.path.pop(0)

Combined all below cells for data and save to CSV in a "preprocess" function

In [2]:
from codes.preprocess import preprocess
preprocess()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


#### Import Libraries

In [2]:
import time
from codes.preprocess import read_csv
import pandas as pd
pd.options.display.max_columns = None

In [3]:
pd.options.display.max_rows = 25

Time to load "Acquisition":  4 to 5 seconds
<br>Time to load "Performance":  40 to 50 seconds

In [3]:
start_time = time.time()
acquisition_df = read_csv('Acquisition')
print(f'Time to load "Acquisition" {round(time.time()-start_time,4)} seconds')

/Users/jleifnf/Desktop/DS Codes/Secondary_Mortgage_Loans/codes


FileNotFoundError: [Errno 2] File b'Acquisition_2018Q1.txt' does not exist: b'Acquisition_2018Q1.txt'

In [4]:
start_time = time.time()
performance_df = read_csv('Performance')
print(f'Time to load "Performance" {round(time.time()-start_time,4)} seconds')

Time to load "Acquisition" 4.8946 seconds
Time to load "Performance" 47.9346 seconds


In [5]:
display(acquisition_df.head(5))
display(performance_df.head(5))

,id,channel,seller,interest_rate,balance,loan_term,origination_date,ltv,cltv,borrower_count,dti,borrower_score,first_time_homebuyer,loan_purpose,property_type,unit_count,occupancy_type,property_state,insurance_percentage,product_type,coborrower_score,insurance_type,relocation_flag
0,100001040173,R,QUICKEN LOANS INC.,4.25,453000,360,01/2018,65,65.0,1,28.0,791.0,N,C,PU,1,P,OH,NaN,FRM,NaN,NaN,N
1,100002370993,C,"WELLS FARGO BANK, N.A.",4.25,266000,360,01/2018,80,80.0,2,41.0,736.0,N,R,PU,1,P,IN,NaN,FRM,793.0,NaN,N
2,100005405807,R,PMTT4,3.99,233000,360,12/2017,79,79.0,2,48.0,696.0,N,R,SF,1,P,CA,NaN,FRM,665.0,NaN,N
3,100008071646,R,OTHER,4.25,184000,360,01/2018,80,80.0,1,48.0,767.0,Y,P,PU,1,P,FL,NaN,FRM,NaN,NaN,N
4,100010739040,R,OTHER,4.25,242000,360,02/2018,49,49.0,1,22.0,727.0,N,R,SF,1,P,CA,NaN,FRM,NaN,NaN,N


,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag
0,100001040173,02/01/2018,4.25,NaN,0,360,360.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
1,100001040173,03/01/2018,4.25,NaN,1,359,359.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
2,100001040173,04/01/2018,4.25,NaN,2,358,358.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
3,100001040173,05/01/2018,4.25,NaN,3,357,357.0,18140,0,N,NaN,NaN,NaN,NaN,NaN
4,100001040173,06/01/2018,4.25,NaN,4,356,356.0,18140,0,N,NaN,NaN,NaN,NaN,NaN


## SQLite DataBase
NOTE: Uncomment to creat a SQLite Database on computer

In [6]:
# import sqlite3
# conn = sqlite3.connect('Secondary_Mortgage_Loans.db')
# acquisition_df

In [7]:
# acquisition_df.to_sql('Acquisition', con=conn)
# performance_df.to_sql('Performance', con=conn)

## EDA

In [8]:
acquisition_df.info()
performance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625195 entries, 0 to 1625194
Data columns (total 23 columns):
id                      1625195 non-null int64
channel                 1625195 non-null object
seller                  1625195 non-null object
interest_rate           1625195 non-null float64
balance                 1625195 non-null int64
loan_term               1625195 non-null int64
origination_date        1625195 non-null object
ltv                     1625195 non-null int64
cltv                    1625194 non-null float64
borrower_count          1625195 non-null int64
dti                     1624702 non-null float64
borrower_score          1623501 non-null float64
first_time_homebuyer    1625195 non-null object
loan_purpose            1625195 non-null object
property_type           1625195 non-null object
unit_count              1625195 non-null int64
occupancy_type          1625195 non-null object
property_state          1625195 non-null object
insurance_percentage    5

#### Filter rows outside of 50 states + DC. (2298 rows)
In both Acquisition and Performance.
- Use Acquisition's Loan ID to filter out rows in Performance.

Drop State Codes with 'PR','GU', and 'VI' from both Acquisition and Performance.

In [9]:
acquisition_df.property_state = acquisition_df.property_state.where(~acquisition_df.property_state.isin(['PR','GU','VI']))
acquisition_df = acquisition_df.dropna(subset=['property_state'])
acquisition_df.shape

(1622897, 23)

In [10]:
print('Before removing: ', performance_df.shape[0])
performance_df.id = performance_df.id.where(performance_df.id.isin(acquisition_df.id))
performance_df = performance_df.dropna(subset=['id'])
print('After removing: ', performance_df.shape[0])

Before removing:  24459263
After removing:  24423998


Change "Delinquency Status" type (string to integers) and change "Reporting Period" as DateTime

In [11]:
performance_df['reporting_period']= pd.to_datetime(performance_df['reporting_period'])

In [12]:
performance_df.delinquency_status = performance_df.delinquency_status.fillna(-99).mask(performance_df.delinquency_status == 'X',-99).astype('int8')

In [13]:
# print(performance_df.delinquency_status.isna().sum(), ' rows of NaN in "Delinquency Status"')
performance_df['delinquency_status'].value_counts()

 0     24022577
-99      222653
 1       131824
 2        21958
 3         8630
 4         5127
 5         3621
 6         2367
 7         1620
 8         1173
 9          797
 10         593
 11         405
 12         270
 13         164
 14          96
 15          60
 16          35
 17          17
 18           7
 19           4
Name: delinquency_status, dtype: int64

In [14]:
performance_df['delinquency_bool'] = performance_df.delinquency_status.map(lambda x: 1 if x > 0 else 0)

In [15]:
performance_df['payment_amounts'] = -performance_df.groupby('id').balance.diff()

Backfill NaN for Balance for the first 5-6 months

In [16]:
performance_df.balance = performance_df.balance.fillna(method='bfill')

In [17]:
performance_df = performance_df[performance_df.loan_age > 0]

In [18]:
performance_df.head(25)

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool
1,1.000010e+11,2018-03-01,4.25,449321.18,1,359,359.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
2,1.000010e+11,2018-04-01,4.25,449321.18,2,358,358.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
3,1.000010e+11,2018-05-01,4.25,449321.18,3,357,357.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
4,1.000010e+11,2018-06-01,4.25,449321.18,4,356,356.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
5,1.000010e+11,2018-07-01,4.25,449321.18,5,355,355.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
6,1.000010e+11,2018-08-01,4.25,449321.18,6,354,354.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
7,1.000010e+11,2018-09-01,4.25,448683.55,7,353,353.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
8,1.000010e+11,2018-10-01,4.25,448043.66,8,352,352.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
9,1.000010e+11,2018-11-01,4.25,447401.50,9,351,351.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
10,1.000010e+11,2018-12-01,4.25,446757.07,10,350,350.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0


In [19]:
# zero_balanced = performance_df[performance_df.zero_balance_code==1]

In [20]:
performance_df[performance_df.zero_balance_code==15]

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool
2618851,3.852559e+11,2019-07-01,4.125,217185.17,19,341,0.0,0,-99,N,15.0,07/2019,02/01/2018,NaN,N,0
4088043,5.465659e+11,2019-07-01,4.125,470007.81,18,342,0.0,35620,-99,N,15.0,07/2019,07/01/2018,NaN,N,0
8700302,1.686090e+11,2019-07-01,3.500,183021.26,15,345,0.0,16980,-99,N,15.0,07/2019,06/01/2018,NaN,N,0
9680629,3.107495e+11,2019-07-01,4.500,251085.72,15,345,0.0,42680,-99,N,15.0,07/2019,06/01/2018,NaN,N,0
11412304,5.631764e+11,2019-07-01,5.500,286409.14,15,345,0.0,46060,-99,N,15.0,07/2019,09/01/2018,NaN,N,0
12715512,7.517985e+11,2019-07-01,5.875,197041.40,14,346,0.0,19100,-99,N,15.0,07/2019,05/01/2018,NaN,N,0
13586744,8.768427e+11,2019-07-01,5.125,185032.51,15,345,0.0,37980,-99,N,15.0,07/2019,05/01/2018,NaN,N,0
14344940,9.866173e+11,2019-07-01,4.500,233605.71,15,345,0.0,18140,-99,N,15.0,07/2019,07/01/2018,NaN,N,0


In [21]:
performance_df[performance_df.balance == 0]

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool
6928525,8.574486e+11,2019-05-01,4.375,0.0,13,347,0.0,44100,-99,N,1.0,05/2019,NaN,NaN,N,0
11274145,5.429086e+11,2019-04-01,4.875,0.0,10,350,0.0,0,-99,N,1.0,04/2019,NaN,NaN,N,0
15191139,2.105369e+11,2019-08-01,3.925,0.0,11,169,0.0,17460,-99,N,6.0,08/2019,NaN,NaN,Y,0


In [22]:
performance_df.zero_balance_code.value_counts()

1.0     220908
6.0       1116
9.0        143
2.0         43
3.0         19
15.0         8
16.0         1
Name: zero_balance_code, dtype: int64

Looking at some delinquency_status data.

Example of 19 months (max number) of Deliquent Loan

In [23]:
performance_df[performance_df.id == 598653679058]

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool
4563838,5.986537e+11,2018-02-01,5.5,142344.02,1,359,359.0,26420,0,N,NaN,NaN,NaN,NaN,NaN,0
4563839,5.986537e+11,2018-03-01,5.5,142344.02,2,358,359.0,26420,1,N,NaN,NaN,NaN,NaN,NaN,1
4563840,5.986537e+11,2018-04-01,5.5,142344.02,3,357,359.0,26420,2,N,NaN,NaN,NaN,NaN,NaN,1
4563841,5.986537e+11,2018-05-01,5.5,142344.02,4,356,359.0,26420,3,N,NaN,NaN,NaN,NaN,NaN,1
4563842,5.986537e+11,2018-06-01,5.5,142344.02,5,355,359.0,26420,4,N,NaN,NaN,NaN,NaN,NaN,1
4563843,5.986537e+11,2018-07-01,5.5,142344.02,6,354,359.0,26420,5,N,NaN,NaN,NaN,NaN,NaN,1
4563844,5.986537e+11,2018-08-01,5.5,142344.02,7,353,359.0,26420,6,N,NaN,NaN,NaN,NaN,NaN,1
4563845,5.986537e+11,2018-09-01,5.5,142344.02,8,352,359.0,26420,7,N,NaN,NaN,NaN,NaN,NaN,1
4563846,5.986537e+11,2018-10-01,5.5,142344.02,9,351,359.0,26420,8,N,NaN,NaN,NaN,NaN,NaN,1
4563847,5.986537e+11,2018-11-01,5.5,142344.02,10,350,359.0,26420,9,N,NaN,NaN,NaN,NaN,NaN,1


### Fully-paid \[zero_balance_code = 1; make_whole_flag (Repurchase Make Whole Proceeds Flag) = 'N'\]
### RealEstateOwned \[zero_balance_code = 9 \]

In [29]:
fully_paid = performance_df[(performance_df.zero_balance_code.isin([1, 9])) & (performance_df.make_whole_flag=='N')]

### Foreclosure \[zero_balance_code = 3,6,15]

In [25]:
defaulted = performance_df[performance_df.zero_balance_code.isin([3,6,15])]

### DELINQUENT

In [36]:
delinq_ids = performance_df[performance_df['delinquency_bool']==1].groupby('id').nth(0).reset_index().id
delinq_ids

0        1.000028e+11
1        1.000316e+11
2        1.000533e+11
3        1.000666e+11
4        1.000754e+11
             ...     
69821    9.999052e+11
69822    9.999309e+11
69823    9.999621e+11
69824    9.999634e+11
69825    9.999714e+11
Name: id, Length: 69826, dtype: float64

In [37]:
delinq_loans = performance_df[performance_df.id.isin(delinq_ids)]

In [51]:
delinq_loans.head(25)

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,foreclosure_writeoff,delinquency_bool
319,1.000316e+11,2018-05-01,4.375,365255.06,1,359,359.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
320,1.000316e+11,2018-06-01,4.375,365255.06,2,358,357.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
321,1.000316e+11,2018-07-01,4.375,365255.06,3,357,357.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
322,1.000316e+11,2018-08-01,4.375,365255.06,4,356,356.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
323,1.000316e+11,2018-09-01,4.375,365255.06,5,355,355.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
324,1.000316e+11,2018-10-01,4.375,364675.66,6,354,354.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
325,1.000316e+11,2018-11-01,4.375,364094.15,7,353,353.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
326,1.000316e+11,2018-12-01,4.375,364094.15,8,352,353.0,16740,1,N,NaN,NaN,NaN,NaN,NaN,1
327,1.000316e+11,2019-01-01,4.375,362999.97,9,351,351.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
328,1.000316e+11,2019-02-01,4.375,362412.35,10,350,350.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0


In [53]:
delinq_loans.tail(25)

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,foreclosure_writeoff,delinquency_bool
24458827,9.999052e+11,2019-07-01,5.25,206035.80,8,352,352.0,36740,0,N,NaN,NaN,NaN,NaN,NaN,0
24458828,9.999052e+11,2019-08-01,5.25,205788.63,9,351,351.0,36740,0,N,NaN,NaN,NaN,NaN,NaN,0
24458829,9.999052e+11,2019-09-01,5.25,205472.43,10,350,350.0,36740,0,N,NaN,NaN,NaN,NaN,NaN,0
24458943,9.999309e+11,2018-10-01,5.99,377306.83,1,359,359.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458944,9.999309e+11,2018-11-01,5.99,377306.83,2,358,358.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458945,9.999309e+11,2018-12-01,5.99,377306.83,3,357,357.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458946,9.999309e+11,2019-01-01,5.99,377306.83,4,356,356.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458947,9.999309e+11,2019-02-01,5.99,377306.83,5,355,355.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458948,9.999309e+11,2019-03-01,5.99,377306.83,6,354,354.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
24458949,9.999309e+11,2019-04-01,5.99,377306.83,7,353,353.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0


In [38]:
before_deliq_rows = delinq_loans[delinq_loans.groupby('id').delinquency_status.diff(-1) == -1].groupby('id').nth(0)

In [58]:
before_deliq_rows

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,foreclosure_writeoff,delinquency_bool
0,1.000028e+11,2018-06-01,4.750,331674.96,2,358,358.0,19100,0,N,NaN,NaN,NaN,NaN,NaN,0
1,1.000316e+11,2018-11-01,4.375,364094.15,7,353,353.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
2,1.000533e+11,2018-12-01,4.625,133990.26,5,355,355.0,0,0,N,NaN,NaN,NaN,NaN,NaN,0
3,1.000666e+11,2019-02-01,5.375,346032.71,3,357,357.0,29820,0,N,NaN,NaN,NaN,NaN,NaN,0
4,1.000754e+11,2019-05-01,4.500,57133.36,12,168,168.0,19740,0,N,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63499,9.999010e+11,2019-05-01,4.375,72710.84,9,171,171.0,38300,0,N,NaN,NaN,NaN,NaN,NaN,0
63500,9.999309e+11,2019-04-01,5.990,377306.83,7,353,353.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
63501,9.999621e+11,2019-04-01,4.125,242303.78,15,345,345.0,12060,0,N,NaN,NaN,NaN,NaN,NaN,0
63502,9.999634e+11,2019-07-01,5.250,276301.17,8,352,352.0,31460,0,N,NaN,NaN,NaN,NaN,NaN,0


<p>behavior of payers up to point of deliquency
<br>(using balance)
</p>

#### Delinq_status = 1, but use the delinq_status = 0 row

In [38]:
before_deliq_rows = performance_df[(performance_df['delinquency_status']==0) & 
                                   (performance_df.id.isin(delinq_ids))].groupby('id').nth(-1)

In [39]:
before_deliq_rows

,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,foreclosure_writeoff,delinquency_bool
id,,,,,,,,,,,,,,,
1.000028e+11,2019-09-01,4.750,326555.37,17,343,341.0,19100,0,N,NaN,NaN,NaN,NaN,NaN,0
1.000316e+11,2019-09-01,4.375,345644.81,17,343,319.0,16740,0,N,NaN,NaN,NaN,NaN,NaN,0
1.000533e+11,2019-09-01,4.625,132187.71,14,346,345.0,0,0,N,NaN,NaN,NaN,NaN,NaN,0
1.000666e+11,2019-09-01,5.375,344020.97,10,350,350.0,29820,0,N,NaN,NaN,NaN,NaN,NaN,0
1.000754e+11,2019-09-01,4.500,56148.84,16,164,164.0,19740,0,N,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.999052e+11,2019-09-01,5.250,205472.43,10,350,350.0,36740,0,N,NaN,NaN,NaN,NaN,NaN,0
9.999309e+11,2019-08-01,5.990,375725.19,11,349,349.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
9.999621e+11,2019-09-01,4.125,240454.16,20,340,340.0,12060,0,N,NaN,NaN,NaN,NaN,NaN,0


In [34]:
first_delinq_rows.head()

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,foreclosure_writeoff,delinquency_bool
0,1.000028e+11,2018-07-01,4.750,331674.96,3,357,358.0,19100,1,N,NaN,NaN,NaN,NaN,NaN,1
1,1.000316e+11,2018-12-01,4.375,364094.15,8,352,353.0,16740,1,N,NaN,NaN,NaN,NaN,NaN,1
2,1.000533e+11,2019-01-01,4.625,133990.26,6,354,355.0,0,1,N,NaN,NaN,NaN,NaN,NaN,1
3,1.000666e+11,2019-03-01,5.375,346032.71,4,356,357.0,29820,1,N,NaN,NaN,NaN,NaN,NaN,1
4,1.000754e+11,2019-06-01,4.500,57133.36,13,167,168.0,19740,1,N,NaN,NaN,NaN,NaN,NaN,1


In [72]:
not_delinq = performance_df[~performance_df.id.isin(last_delinq_rows.id)]

In [74]:
defaulted = performance_df[~performance_df.foreclosure_date.isna()]

8.557721e+11    1
8.746417e+11    1
7.807047e+11    1
9.838986e+11    1
6.932074e+11    1
               ..
6.950050e+11    1
7.402484e+11    1
7.957777e+11    1
6.756232e+11    1
5.986189e+11    1
Name: id, Length: 180, dtype: int64

### Current \[delinquency_status = 0]

In [46]:
current_loans = performance_df[performance_df.delinquency_status==0].groupby('id').nth(-1).reset_index()

In [47]:
current_loans.shape

(1619766, 16)

In [48]:
current_loans[~current_loans.id.isin(delinq_ids) | ~current_loans.id.isin(defaulted) | ~current_loans.id.isin(
        fully_paid)]

,id,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,make_whole_flag,delinquency_bool
0,1.000003e+11,2019-09-01,4.250,262004.04,15,345,336.0,19780,0,N,NaN,NaN,NaN,NaN,NaN,0
1,1.000010e+11,2019-09-01,4.250,440853.48,19,341,341.0,18140,0,N,NaN,NaN,NaN,NaN,NaN,0
2,1.000016e+11,2019-03-01,4.625,337063.04,11,349,349.0,38060,0,N,NaN,NaN,NaN,NaN,NaN,0
3,1.000016e+11,2019-08-01,4.625,446559.21,11,349,349.0,16980,0,N,NaN,NaN,NaN,NaN,NaN,0
4,1.000022e+11,2019-09-01,5.000,84065.32,8,352,351.0,26420,0,N,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619761,9.999979e+11,2019-09-01,4.375,279782.33,12,348,348.0,22220,0,N,NaN,NaN,NaN,NaN,NaN,0
1619762,9.999985e+11,2019-09-01,5.500,392477.89,16,344,343.0,41740,0,N,NaN,NaN,NaN,NaN,NaN,0
1619763,9.999985e+11,2019-09-01,4.875,67477.76,17,283,282.0,37460,0,N,NaN,NaN,NaN,NaN,NaN,0
1619764,9.999988e+11,2019-09-01,4.250,348713.11,19,341,341.0,39900,0,N,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
1577833 -

In [91]:
performance_df[performance_df.delinquency_status==0].describe().T

,count,mean,std,min,25%,50%,75%,max
id,14398871.0,5.499711e+11,2.599987e+11,1.000003e+11,3.246723e+11,5.503851e+11,7.754477e+11,9.999992e+11
interest_rate,14398871.0,4.562214e+00,5.348707e-01,2.500000e+00,4.250000e+00,4.625000e+00,4.875000e+00,6.875000e+00
balance,14398871.0,2.239685e+05,1.195839e+05,1.000000e-02,1.325476e+05,2.021948e+05,2.947096e+05,1.299611e+06
loan_age,14398871.0,1.097701e+01,3.702738e+00,3.000000e+00,8.000000e+00,1.000000e+01,1.400000e+01,8.100000e+01
months_to_maturity,14398871.0,3.186929e+02,6.600519e+01,4.200000e+01,3.430000e+02,3.480000e+02,3.520000e+02,4.820000e+02
adj_months_to_maturity,14397555.0,3.146894e+02,6.909399e+01,1.000000e+00,3.390000e+02,3.470000e+02,3.510000e+02,4.800000e+02
msa,14398871.0,2.754443e+04,1.359784e+04,0.000000e+00,1.698000e+04,3.108000e+04,3.890000e+04,4.974000e+04
delinquency_status,14398871.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
zero_balance_code,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
delinquency_bool,14398871.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [102]:
current_loans = current_loans[~current_loans.id.isin(last_delinq_rows.id)]
current_loans = current_loans[~current_loans.id.isin(defaulted.id)]
current_loans.groupby('id').nth(-1)

,reporting_period,interest_rate,balance,loan_age,months_to_maturity,adj_months_to_maturity,msa,delinquency_status,modification_flag,zero_balance_code,zero_balance_date,last_paid_installment_date,foreclosure_date,delinquency_bool,payment_amounts
id,,,,,,,,,,,,,,,
1.000003e+11,2019-09-01,4.250,262004.04,15,345,336.0,19780,0,N,NaN,NaN,NaN,NaN,0,-639.76
1.000010e+11,2019-09-01,4.250,440853.48,19,341,341.0,18140,0,N,NaN,NaN,NaN,NaN,0,-665.27
1.000016e+11,2019-03-01,4.625,337063.04,11,349,349.0,38060,0,N,NaN,NaN,NaN,NaN,0,-457.50
1.000016e+11,2019-08-01,4.625,446559.21,11,349,349.0,16980,0,N,NaN,NaN,NaN,NaN,0,0.00
1.000022e+11,2019-09-01,5.000,84065.32,8,352,351.0,26420,0,N,NaN,NaN,NaN,NaN,0,-105.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.999979e+11,2019-09-01,4.375,279782.33,12,348,348.0,22220,0,N,NaN,NaN,NaN,NaN,0,-398.85
9.999985e+11,2019-09-01,5.500,392477.89,16,344,343.0,41740,0,N,NaN,NaN,NaN,NaN,0,-471.29
9.999985e+11,2019-09-01,4.875,67477.76,17,283,282.0,37460,0,N,NaN,NaN,NaN,NaN,0,-127.75
